In [17]:
import os

import fiftyone as fo
import fiftyone.brain as fob
import fiftyone.zoo as foz
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import shutil

ROOT_IMG_DIR = '/home/gpubox2/plantin-projects/coins/test-coin-service/user_images'

In [18]:
data_df = pd.read_csv('../api-service/user_ann-test.csv')

In [21]:
data_df.groupby('task_alias').first()

,task_file_name,task_state,task_coin_side,task_created_at,task_updated_at,category_id,original_url,coin_side,img_id,file_name,name,denomination,year,country
task_alias,,,,,,,,,,,,,,
usa-obverse-0,a7f1189f-951d-4c60-9c13-f4beb30f5a97.jpg,annotated,obverse,2023-03-30 13:59:00.303,2023-03-30 15:10:30.843,680,https://i.ucoin.net/coin/7/304/7304150-1s/usa-...,obverse,7105,7105.jpg,USA 1 cent,1 cent,1959-1982,USA
usa-obverse-10,3da0f06b-a2fa-4d24-9d07-cc56c7db1359.jpg,annotated,obverse,2023-03-30 13:59:02.710,2023-03-30 23:21:32.734,690,https://i.ucoin.net/coin/41/227/41227258-2s/us...,reverse,7346,7346.jpg,USA 1 dollar,1 dollar,2000-2008,USA
usa-obverse-3,437c0f6b-2e71-4d84-8d9d-d7395d61e11e.jpg,annotated,obverse,2023-03-30 13:59:00.303,2023-03-30 14:59:47.459,683,https://i.ucoin.net/coin/1/182/1182953-1s/usa-...,obverse,7177,7177.jpg,USA 5 cents,5 cents,1946-2003,USA
usa-obverse-6,fbe90c19-3551-4d46-846c-bf365927c4fc.jpg,annotated,obverse,2023-03-30 13:59:00.387,2023-03-30 18:03:34.465,686,https://i.ucoin.net/coin/22/654/22654386-1s/us...,obverse,7249,7249.jpg,USA ¼ dollar,¼ dollar,1965-1998,USA
usa-obverse-62,662b196c-34c5-406a-bf44-59d5e8983a99.jpg,annotated,obverse,2023-03-30 13:59:02.715,2023-03-31 00:44:32.185,36816,https://i.ucoin.net/coin/48/313/48313595-1s/us...,obverse,342678,342678.jpg,USA ¼ dollar,¼ dollar,1999,USA


In [20]:
data_df['task_file_name'].unique()

array(['437c0f6b-2e71-4d84-8d9d-d7395d61e11e.jpg',
       'a7f1189f-951d-4c60-9c13-f4beb30f5a97.jpg',
       '501991ef-2234-4feb-aebb-84666a01a3c7.jpg',
       '52377f83-b01b-4d23-86c2-25529fa30aa1.jpg',
       '3cff9eae-d8aa-4035-9321-ed264fdb50f1.jpg',
       '7cefcb04-74fc-4e09-9aae-7c4e10ce3e9c.jpg',
       'c5c6e56b-3ffb-42ff-ae43-20b8ee3f10c9.jpg',
       'fbe90c19-3551-4d46-846c-bf365927c4fc.jpg',
       '0d09bd2d-f767-4da8-a3df-da2ca0d810a4.jpg',
       '3da0f06b-a2fa-4d24-9d07-cc56c7db1359.jpg',
       '662b196c-34c5-406a-bf44-59d5e8983a99.jpg'], dtype=object)

In [16]:
data_df.sample(10)



,task_alias,task_file_name,task_state,task_coin_side,task_created_at,task_updated_at,category_id,original_url,coin_side,img_id,file_name,name,denomination,year,country
160,usa-obverse-0,7cefcb04-74fc-4e09-9aae-7c4e10ce3e9c.jpg,annotated,obverse,2023-03-30 13:59:00.383,2023-03-30 17:57:12.688,691,https://i.ucoin.net/coin/22/804/22804095-1s/us...,obverse,7381,7381.jpg,USA 1 cent,1 cent,1944-1958,USA
69,usa-obverse-0,501991ef-2234-4feb-aebb-84666a01a3c7.jpg,annotated,obverse,2023-03-30 13:59:00.304,2023-03-30 15:12:58.099,691,https://i.ucoin.net/coin/22/804/22804336-1s/us...,obverse,7387,7387.jpg,USA 1 cent,1 cent,1944-1958,USA
266,usa-obverse-62,662b196c-34c5-406a-bf44-59d5e8983a99.jpg,annotated,obverse,2023-03-30 13:59:02.715,2023-03-31 00:44:32.185,36820,https://i.ucoin.net/coin/6/987/6987933-1s/usa-...,obverse,342758,342758.jpg,USA ¼ dollar,¼ dollar,2016,USA
36,usa-obverse-0,a7f1189f-951d-4c60-9c13-f4beb30f5a97.jpg,annotated,obverse,2023-03-30 13:59:00.303,2023-03-30 15:10:30.843,691,https://i.ucoin.net/coin/22/804/22804107-1s/us...,obverse,7369,7369.jpg,USA 1 cent,1 cent,1944-1958,USA
178,usa-obverse-6,fbe90c19-3551-4d46-846c-bf365927c4fc.jpg,annotated,obverse,2023-03-30 13:59:00.387,2023-03-30 18:03:34.465,686,https://i.ucoin.net/coin/22/654/22654386-1s/us...,obverse,7249,7249.jpg,USA ¼ dollar,¼ dollar,1965-1998,USA
217,usa-obverse-10,3da0f06b-a2fa-4d24-9d07-cc56c7db1359.jpg,annotated,obverse,2023-03-30 13:59:02.710,2023-03-30 23:21:32.734,690,https://i.ucoin.net/coin/0/100/100962-2s/usa-1...,reverse,7352,7352.jpg,USA 1 dollar,1 dollar,2000-2008,USA
189,usa-obverse-6,fbe90c19-3551-4d46-846c-bf365927c4fc.jpg,annotated,obverse,2023-03-30 13:59:00.387,2023-03-30 18:03:34.465,686,https://i.ucoin.net/coin/22/655/22655269-1s/us...,obverse,7271,7271.jpg,USA ¼ dollar,¼ dollar,1965-1998,USA
39,usa-obverse-0,a7f1189f-951d-4c60-9c13-f4beb30f5a97.jpg,annotated,obverse,2023-03-30 13:59:00.303,2023-03-30 15:10:30.843,691,https://i.ucoin.net/coin/22/804/22804402-1s/us...,obverse,7375,7375.jpg,USA 1 cent,1 cent,1944-1958,USA
3,usa-obverse-3,437c0f6b-2e71-4d84-8d9d-d7395d61e11e.jpg,annotated,obverse,2023-03-30 13:59:00.303,2023-03-30 14:59:47.459,683,https://i.ucoin.net/coin/23/175/23175595-1s/us...,obverse,7183,7183.jpg,USA 5 cents,5 cents,1946-2003,USA
121,usa-obverse-0,3cff9eae-d8aa-4035-9321-ed264fdb50f1.jpg,annotated,obverse,2023-03-30 13:59:00.325,2023-03-30 16:12:06.079,691,https://i.ucoin.net/coin/22/804/22804402-1s/us...,obverse,7375,7375.jpg,USA 1 cent,1 cent,1944-1958,USA


In [15]:
for _, row in data_df.iterrows():
    img_path = os.path.join(ROOT_IMG_DIR, row['task_file_name'])
    if not os.path.exists(img_path):
        print(f"Image {img_path} does not exist")
        # continue
        
    shutil.copy(img_path, f"../data/user-images/{row['task_file_name']}")
    
   
